# Clustering Crypto

In [123]:
# Initial imports
import requests
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [124]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [125]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [126]:
# Alternatively, use the provided csv file:

file_path = Path("crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path)

# Display DataFrame
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [127]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.drop(columns=['Unnamed: 0'])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [128]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [129]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [130]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns=['IsTrading'])
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [131]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [132]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [133]:
# Drop rows where there are 'N/A' text values
for row in crypto_df:
    crypto_df = crypto_df.loc[crypto_df[row] != 'N/A']

crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000
10,Dash,X11,PoW/PoS,9.031294e+06,22000000
11,Monero,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
13,ZCash,Equihash,PoW,7.383056e+06,21000000


In [134]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = crypto_df[['CoinName']]
coins_name.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [135]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns=['CoinName'])
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [136]:
# Create dummy variables for text features
text_feature_variable = ''

# Import Label Encoder from SKLearn and encode categorical data:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(crypto_df['Algorithm'])
crypto_df['Algorithm'] = le.transform(crypto_df['Algorithm'])

le.fit(crypto_df['ProofType'])
crypto_df['ProofType'] = le.transform(crypto_df['ProofType'])


crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,53,16,4.199995e+01,42
2,53,16,1.055185e+09,532000000
5,67,16,2.927942e+10,314159265359
7,47,13,1.792718e+07,21000000
8,20,13,1.076842e+08,0


In [137]:
# Standardize data
scaler = StandardScaler()

df_scaled = scaler.fit_transform(crypto_df)
df_scaled

array([[ 0.39335561,  0.8871567 , -0.11674788, -0.15286468],
       [ 0.39335561,  0.8871567 , -0.09358885, -0.14499604],
       [ 1.21439604,  0.8871567 ,  0.52587231,  4.4937636 ],
       ...,
       [-2.12841143,  0.00878917, -0.09523411, -0.13215444],
       [-1.65924547,  0.00878917, -0.11658774, -0.15255408],
       [ 0.39335561, -1.45515672, -0.11674507, -0.15284989]])

### Reducing Dimensions Using PCA

In [142]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [143]:
# Create a DataFrame with the principal components data
crypto_pca = pca.fit_transform(df_scaled)

pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)

pcs_df.head()

,PC 1,PC 2,PC 3
0,-0.412665,0.823150,0.357227
2,-0.391306,0.827873,0.358336
5,3.143152,2.186748,0.487273
7,-0.190301,-0.003829,-0.043688
8,-0.051013,-1.161387,1.020473


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [144]:
inertia = []
k = list(range(1, 11))

# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [145]:
# Initialize the K-Means model
model = KMeans(n_clusters=6, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = df.join(pcs_df)
clustered_df['CoinName'] = coins_name['CoinName']
clustered_df['Class'] = predictions
clustered_df.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,53,16,4.199995e+01,42,-0.412665,0.823150,0.357227,42 Coin,0
2,53,16,1.055185e+09,532000000,-0.391306,0.827873,0.358336,404Coin,0
5,67,16,2.927942e+10,314159265359,3.143152,2.186748,0.487273,EliteCoin,5
7,47,13,1.792718e+07,21000000,-0.190301,-0.003829,-0.043688,Bitcoin,4
8,20,13,1.076842e+08,0,-0.051013,-1.161387,1.020473,Ethereum,1


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [146]:
# Scale data to create the scatter plot
#scaler = StandardScaler()

#clustered_df_scaled = scaler.fit_transform(clustered_df)
#clustered_df_scaled

In [147]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
    width=800,
    height=500
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [148]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [149]:
# Print the total number of tradable cryptocurrencies
len(clustered_df.CoinName)

533